# Training Linear Models

In [9]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
from pathlib import Path
from sklearn.preprocessing import add_dummy_feature, PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso, ElasticNet, LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import learning_curve, train_test_split
from sklearn.metrics import root_mean_squared_error
from math import ceil